# Training YOLOv5 custom dataset with ease

Check the full explanation on medium:

https://medium.com/mlearning-ai/training-yolov5-custom-dataset-with-ease-e4f6272148ad

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 16867, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 16867 (delta 14), reused 28 (delta 4), pack-reused 16825 (from 1)
Receiving objects: 100% (16867/16867), 15.60 MiB | 11.74 MiB/s, done.
Resolving deltas: 100% (11563/11563), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 805.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.0/869.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
!gdown https://drive.google.com/uc?id=1YTqFZZ2oNW9qwY4SCkgZh2Jwt6BOmhCt

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/__main__.py", line 163, in main
    download(
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 188, in download
    res = sess.get(url, stream=True, verify=verify)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 602, in get
    return self.request("GET", url, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 589, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 703, in send
    r = adapter.send(request, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/adapters.py", line 667, in send
    resp = conn.urlopen(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import pandas as pd
import cv2 as cv
import numpy as np
from skimage.feature import hog
from yolov5 import utils, train, val
import os
import shutil
import random
import sys
display = utils.notebook_init()

YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.6/78.2 GB disk)


In [ ]:
zip_file = "path to dataset"

if os.path.isfile(zip_file):
  shutil.unpack_archive(zip_file, "data")
else:
  print(zip_file + " not found")

In [ ]:
##Use this function for HSV with HOG preprocessing

def preprocessing(image):
  image = (image - np.min(image))/np.ptp(image) #normalize image
  image = (image * 255).astype(np.uint8)
  image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
  hog_features = extract_hog_features(image[:, :, 2])  # V channel
  # # Reshape HOG features to match original image size
  hog_image = hog_features.reshape(image[:, :, 2].shape)
  # # Stack HOG with H and S channels to form a 3-channel image
  image = np.stack([image[:, :, 0], image[:, :, 1], hog_image], axis=-1)

  return image

def extract_hog_features(image):
  # Use skimage's hog function to extract HOG features
  features, hog_image = hog(image, pixels_per_cell=(8, 8),
                            cells_per_block=(2, 2), visualize=True)
  return hog_image


def save_preprocessed_image(src_image_path, dest_image_path):
    image = cv.imread(src_image_path)
    preprocessed_image = preprocessing(image)
    cv.imwrite(dest_image_path, preprocessed_image)

In [ ]:
##Use this function for RGB preprocessing

def preprocessing(image):
  image = (image - np.min(image))/np.ptp(image) #normalize image
  image = (image * 255).astype(np.uint8)
  return image

def save_preprocessed_image(src_image_path, dest_image_path):
    image = cv.imread(src_image_path)
    preprocessed_image = preprocessing(image)
    cv.imwrite(dest_image_path, preprocessed_image)

In [ ]:
# Preparing the folder structure

full_data_paths = ['/content/yolov5/data/rupiah-banknotes/1000/',
                   '/content/yolov5/data/rupiah-banknotes/2000/',
                   '/content/yolov5/data/rupiah-banknotes/5000/',
                   '/content/yolov5/data/rupiah-banknotes/10000/',
                   '/content/yolov5/data/rupiah-banknotes/20000/',
                   '/content/yolov5/data/rupiah-banknotes/50000/',
                   '/content/yolov5/data/rupiah-banknotes/75000/',
                   '/content/yolov5/data/rupiah-banknotes/100000/']
extension_allowed = '.jpg'
split_percentage = 80

images_path = 'data/images/'
if os.path.exists(images_path):
    shutil.rmtree(images_path)
os.mkdir(images_path)

labels_path = 'data/labels/'
if os.path.exists(labels_path):
    shutil.rmtree(labels_path)
os.mkdir(labels_path)

training_images_path = os.path.join(images_path, 'training/')
test_images_path = os.path.join(images_path, 'test/')
training_labels_path = os.path.join(labels_path, 'training/')
test_labels_path = os.path.join(labels_path, 'test/')

os.mkdir(training_images_path)
os.mkdir(test_images_path)
os.mkdir(training_labels_path)
os.mkdir(test_labels_path)

files = []

ext_len = len(extension_allowed)

# Loop through each path
for full_data_path in full_data_paths:
    for r, d, f in os.walk(full_data_path):
        for file in f:
            if file.endswith(extension_allowed):
                strip = file[0:len(file) - ext_len]
                files.append((full_data_path, strip))

random.shuffle(files)

size = len(files)

split = int(split_percentage * size / 100)

print("copying training data")
for i in range(split):
    full_data_path, strip = files[i]

    image_file = strip + extension_allowed
    src_image = os.path.join(full_data_path, image_file)
    dest_image = os.path.join(training_images_path, image_file)
    save_preprocessed_image(src_image, dest_image)

    annotation_file = strip + '.txt'
    src_label = os.path.join(full_data_path, annotation_file)
    dest_label = os.path.join(training_labels_path, annotation_file)
    shutil.copy(src_label, dest_label)

print("copying test data")
for i in range(split, size):
    full_data_path, strip = files[i]

    image_file = strip + extension_allowed
    src_image = os.path.join(full_data_path, image_file)
    dest_image = os.path.join(test_images_path, image_file)
    save_preprocessed_image(src_image, dest_image)

    annotation_file = strip + '.txt'
    src_label = os.path.join(full_data_path, annotation_file)
    dest_label = os.path.join(test_labels_path, annotation_file)
    shutil.copy(src_label, dest_label)

print("finished")


copying training data
copying test data
finished


In [ ]:
f = open("dataset.yaml", "a")

f.write("train: ../data/images/training/\n")
f.write("val: ../data/images/test/\n")
f.write("nc: 8\n")
f.write("names: ['1000', '2000', '5000', '10000', '20000', '50000', '75000', '100000']\n")
f.close()

In [ ]:
%cd yolov5
!python train.py --img 640 --batch 16 --epochs 100 --data /content/yolov5/dataset.yaml --weights yolov5s.pt --device 0

[Errno 2] No such file or directory: 'yolov5'
/content/yolov5
2024-06-05 18:31:18.029546: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 18:31:18.029596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 18:31:18.030778: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, ca

In [ ]:
%cd yolov5
!python val.py --data /content/yolov5/dataset.yaml --weights runs/train/exp/weights/best.pt --img 640 --device 0

[Errno 2] No such file or directory: 'yolov5'
/content/yolov5
val: data=/content/yolov5/dataset.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-318-gc0380fd8 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/yolov5/data/labels/test.cache... 224 images, 0 backgrounds, 0 corrupt: 100% 224/224 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [00:07<00:00,  1.09s/it]
                   all        224        224      0.993      0.981      0.995      0.919
                  1000        224         30      0

In [ ]:
# Plot the training results
from PIL import Image
import matplotlib.pyplot as plt
result_image_path = 'runs/train/exp/results.png'
if os.path.exists(result_image_path):
    img = Image.open(result_image_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
else:
    print("Result image not found!")

In [ ]:
!gdown https://drive.google.com/uc?id=1GpvqhAb6S_ZfbC4VEDjEinA-OUK_TlRT #model download

Downloading...
From: https://drive.google.com/uc?id=1GpvqhAb6S_ZfbC4VEDjEinA-OUK_TlRT
To: /content/yolov5/best.pt
100% 14.5M/14.5M [00:00<00:00, 157MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=1nCndeWA9589R_puhKQBgmopic-u-xy3D

Downloading...
From: https://drive.google.com/uc?id=1nCndeWA9589R_puhKQBgmopic-u-xy3D
To: /content/yolov5/data_test.zip
100% 25.7M/25.7M [00:00<00:00, 210MB/s]


In [ ]:
!python detect.py --weights "Path to model" --img 640 --conf 0.1 --source "path to datatest"

detect: weights=['/content/drive/MyDrive/Final_Project_Stuff/model/yolo_rgb/best.pt'], source=/content/hjhw9u10tddpz9t.jpeg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /content/hjhw9u10tddpz9t.jpeg: 384x640 1 2000, 48.7ms
Speed: 0.7ms pre-process, 48.7ms inference, 763.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp4
